In [1]:
import tensorflow as tf

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e8dfc85d5ed341da82283ca49cb9bdd3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='v0JA25Phl69t1bogqAV0ntkexUT2mouIu0Mz0HO4J1FQ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_3 = client_e8dfc85d5ed341da82283ca49cb9bdd3.get_object(Bucket='breastcancerprediction-donotdelete-pr-txfsfqrcgjjco9', Key='breastcancerdataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
pwd

'/home/wsuser/work'

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range =0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
import os
filenames = os.listdir("/home/wsuser/work/breastcancerdataset/train")

In [7]:
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/breastcancerdataset/train", target_size=(64,64),batch_size=32,class_mode='binary')
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/breastcancerdataset/test",target_size=(64,64),batch_size=32,class_mode='binary')

Found 103 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.layers import MaxPooling2D

In [9]:
model=Sequential()

In [10]:
model.add(Conv2D(64,(3, 3),activation='relu', input_shape=(75, 75, 3)))

In [11]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [12]:
model.add(Flatten())

In [13]:
model.add(Dense(units= 40 ,kernel_initializer='random_uniform',activation = 'relu'))

In [14]:
model.add(Dense(units= 1,activation = 'softmax',kernel_initializer= 'uniform'))

In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.save('breastcancer.h5')

In [17]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

In [18]:
model =  load_model("breastcancer.h5")

In [19]:
!tar -zcvf image_classification_model_new.tgz breastcancer.h5

breastcancer.h5


In [20]:
ls -1

breastcancerdataset/
breastcancer.h5
image_classification_model_new.tgz


In [21]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 14.7 MB/s eta 0:00:01


In [22]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"OVTGh0MhfI-mX0whxXPdi-jec829exvxYRR1vm7OsLFf"
}
client = APIClient(wml_credentials)

In [23]:
client = APIClient(wml_credentials)

In [24]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item["entity"]["name"] == space_name)['metadata']['id'])

In [25]:
space_uid = guid_from_space_name(client, 'cancerimageclassification')
print("space UID = " + space_uid)

space UID = f3fcadd3-c996-4c87-8c7d-ed5bb8770df8


In [26]:
client.set.default_space(space_uid)

'SUCCESS'

In [27]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [32]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [34]:
model_details = client.repository.store_model(model='image_classification_model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                        )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1452: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [35]:
model_id

'472dcef9-b705-4d0c-b2c4-e2d3c5daa718'

In [36]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [37]:
from keras.models import load_model
from keras.preprocessing import image